# VAE Training - Faces dataset

## imports

In [16]:
import os, time
from glob import glob
import numpy as np

from model.VariationalAutoencoder import VariationalAutoencoder
from keras.preprocessing.image import ImageDataGenerator

from keras.preprocessing.image import ImageDataGenerator, load_img, save_img, img_to_array, array_to_img

# environment variables

In [17]:
MODEL_FOLDER = os.environ.get('HOME') + '/model1'
FILE_MODEL = 'weights.h5'

# VAE

## model variables

In [18]:
vae = VariationalAutoencoder()
vae.model.load_weights(os.path.join(MODEL_FOLDER, 'weights.h5'))  

In [21]:
from PIL import Image
from PIL.ImageShow import show

image_path = '/Volumes/My Passport/PFM/output/training_png/Week9_39301_G11_4_Cells_34.png'

img = Image.open(image_path)
array_img  = img_to_array(img) # keras

array_img = array_img / 256.

#print(array_img)

img.show()

print('image format: ' + str(img.format))
#print('image shape: ' + str(img.shape))
print('image mode: ' + img.mode)
print('image size: ' + str(img.size))
print('image width: ' + str(img.width))
print('image height: ' + str(img.height))
print('image palette: ' + str(img.palette))
print('image info: ' + str(img.info))

print('image to array: ' + str(array_img.shape))

array_img = np.expand_dims(array_img, axis=0)


vae.encoder.summary()

print('image to array after expand: ' + str(array_img.shape))

z_result = vae.encoder.predict(array_img)
reco = vae.decoder.predict(z_result).squeeze()

#print(reco)

print(reco.shape)

# scaled
outimg = array_to_img(reco)

show(outimg)

print(reco)


image format: PNG
image mode: RGB
image size: (128, 128)
image width: 128
image height: 128
image palette: None
image info: {}
image to array: (128, 128, 3)
Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
encoder_conv_1 (Conv2D)         (None, 64, 64, 32)   896         encoder_input[0][0]              
__________________________________________________________________________________________________
leaky_re_lu_5 (LeakyReLU)       (None, 64, 64, 32)   0           encoder_conv_1[0][0]             
__________________________________________________________________________________________________
encoder_conv_2 (Conv2D)         (N